# The Hendricks Team

# Using summix_py, An Example Notebook

## Example 1: Solving an Ancestral Deconvolution Problem


* We use simulated SNPs with minor allele frequencies for $K=5$ ancestries -- European ancestries, African ancestries, South Asian ancestries, East Asian ancestries, and Indigenous American ancestires. 

* We numerically solve for the 5 ancestry's true proportions, which is the vector $\pi^*:=(\pi_1,\pi_2,\pi_3,\pi_4, \pi_5)$, in an _observed_ population of our choice.
    * $\pi_1$ denotes the proportion of European ancestries in the observed population
    * $\pi_2$ denotes the proportion of African ancestries in the observed population
    * $\pi_3$ denotes the proportion of South Asian ancestries in the observed population
    * $\pi_4$ denotes the proportion of East Asian ancestries in the observed population
    * $\pi_5$ denotes the proportion of Indigenous American ancestries in the observed population

* We work with an example data set $D$ with $N=10,000$ SNPs and use this data to solve for the ancestry proportions 
    * In short: we minimize MSE between the data and a mixture model using Sequential Least Squares Quadratic Programming, or SLSQP. Please see [The Hendricks Team's paper on bioRxiv](https://www.biorxiv.org/content/10.1101/2021.02.03.429446v1) for mathematical/statistical details.

* The cell below calls the two main functions provided in this package, summix and adjAF (for adjusting allele frequencies)
    * Then we can access the functions inside of the script to solve an example problem.

In [1]:
from summix.summix import SUMMIX
from summix.adj_af import adjAF
import numpy as np

* In the cell below, we read in the example data set provided with this package, called "HA-package-data.txt".
    * We have provided this data in a CSV format as well
* We use Pandas to convert from the .txt, tab-delimited format to an array endowed with numerical linear algebra properties Python understands.
* How data is formatted matters:
    * The data should contain Chromosome number, SNP number (RSID, location on genome), base pair (bP), reference and alternate alleles (A1 and A2), the $K$ minor allele frequencies of the reference ancestires, and finally, any observed allele frequencies we wish to model. 
    * $D$ does not contain an indexing column -- Pandas/Jupyter is just showing this as part of the print statement -- it is not a column of $D$ itself. 
    * Notice we only need certain columns of $D$ to solve our example problem
        * That is, we only need the minor allele frequencies and whichever observed allele frequency we are modeling, which should be $K+1$ columns of D. In our case, we need 6 of the total 13 columns.
    * For more info about data formatting, please see our paper.

#### Note that everything done in the cell below is already handled in our HA_script; the user does not have to read-in the data on their own.
###### We choose to show the Pandas read-in process below for general demonstration and checks below. Also, we can print out the data for inspection.

In [2]:
import pandas as pd
# Read in the data (if tab-delimited txt)
D = pd.read_csv("example-data/example-data.txt",sep='\t')

# Read in the data (if CSV)
#D = pd.read_csv("example-data/example-data.csv")

D.head(5) ### Look at the first 5 rows

,CHR,RSID,BP,A1,A2,ref_eur_1000G,ref_afr_1000G,ref_sas_1000G,ref_eas_1000G,ref_iam_1000G,gnomAD_afr,gnomAD_amr,gnomAD_oth
0,1,rs2887286,1156131,C,T,0.173275,0.541663,0.531712,0.846223,0.7093,0.488610,0.525943,0.229705
1,1,rs41477744,2329564,A,G,0.001238,0.035714,0.000000,0.000000,0.0000,0.045914,0.001179,0.008272
2,1,rs9661525,2952840,G,T,0.168316,0.120048,0.099180,0.393853,0.2442,0.135977,0.286052,0.155617
3,1,rs2817174,3044181,C,T,0.428213,0.959325,0.639072,0.570454,0.5000,0.854879,0.488180,0.470425
4,1,rs12139206,3504073,T,C,0.204215,0.801565,0.393671,0.389881,0.3372,0.724178,0.295508,0.258748


* In the cell below, we specify the number of reference ancestries, $K$, (here we have 5) and choose an initial iterate.
    * Choosing an initial iterate is optional.
    * The default initial iterate is $\pi^{(0)}=\frac{1}{K}(1,\ldots,1)\in \mathbb{R}^{K}$, if one is not provided.
    * The initial iterate must be a $K \times 1$ (column) or $1 \times K$ (row) vector (the HA script can handle either shape).

In [3]:
k = 5 # User must specify number of ancestries!
pi_0 = [[0.3,0.1,0.2,0.1,0.3]] # You do not have to provide the initial iterate, but you may.

* In the cell below, we quickly check that we have specified a data matrix $D$ and total number of ancestries $K$ that match the number of SNPs we think we are working with as well as the correct reference ancestry number.

In [4]:
print('our problem includes', np.shape(D)[0], 'SNPs, and', k, 'reference ancestries.')

our problem includes 10000 SNPs, and 5 reference ancestries.


* In the cell below, we specify the required inputs and solve our example deconvolution problem. In detail, our inputs are: 
    * the $K=5$ reference ancestries (their column names as strings)
    * the chosen observed population (its column name as a string)
    * the file name of the data (as a string)
    * $K$ (as an integer, in our case, this is $5$)
    * $\pi_0$ (as a vector, optional and often just a guess)
    * the file format (as a string -- options are 'tab' or 'csv')


In [5]:
# Call the main function of the script, with correctly specified inputs, to obtain an answer_obj, or answer object, to store
# which will store the key outputs for calculations/checks (below)
# print statements will be automatically triggered

my_ref = ['ref_eur_1000G','ref_afr_1000G','ref_sas_1000G','ref_eas_1000G','ref_iam_1000G']
answer_obj = SUMMIX(ref = my_ref,
                    obs = 'gnomAD_afr', 
                    file_name = "example-data/example-data.txt", 
                    k = 5, 
                    guess = pi_0, 
                    file_format = 'tab')

Numerical solution via SLSQP, pi_final =  [0.15761182 0.82514076 0.00327338 0.00762932 0.00634473] 
 
 using observed population: gnomAD_afr 
 
 Number of SLSQP iterations: 10 
 
 Runtime: 0.04575714602833614 seconds 
 
 
 Detailed results:
0.15761181517545278 is the estimated proportion of ref_eur_1000G 

0.825140757921369 is the estimated proportion of ref_afr_1000G 

0.0032733806562866537 is the estimated proportion of ref_sas_1000G 

0.007629315359773101 is the estimated proportion of ref_eas_1000G 

0.0063447308871184644 is the estimated proportion of ref_iam_1000G 



#### _Above we see a printout of the numerical solution, $\pi^{final}$, the name of the chosen observed population, the number of SLSQP iterations, and the time in seconds of the run._


* The numerical solution we have found is given by $$\pi^\text{final}\approx (0.158,0.825,0.003,0.008,0.006)$$

    * $\pi_1^\text{final}\approx 0.158$ denotes the proportion of European ancestries in the observed population
    * $\pi_2^\text{final}\approx 0.825$ denotes the proportion of African ancestries in the observed population
    * $\pi_3^\text{final}\approx 0.003$ denotes the proportion of South Asian ancestries in the observed population
    * $\pi_4^\text{final}\approx 0.008$ denotes the proportion of East Asian ancestries in the observed population
    * $\pi_5^\text{final}\approx 0.006$ denotes the proportion of Indigenous American ancestries in the observed population
    

* Recall that we chose the gnomAD African sample for our observed population in this example.
    
    
* SLSQP should use about 10-12 iterations to obtain this numerical solution
    

* The runtime of the script/computational process should be less than a second on most machines

<br>

* Our "answer_obj" can now be "unpacked" by calling the 0th, 1st, or 2nd entry in that object which will return the solution vector, number of iterations, and time, respectively.
    * May be useful if you want to analyze the solution vector in some fashion, or find average run times over many trials, etc.
    * To demonstrate, let's check that our solution vector adds up to 1 exactly.

In [6]:
pi_hat = answer_obj[0] # Defines pi_final as the solution vector
print(np.sum(pi_hat,axis=0)) # Check that components of solution vector add to 1

1.0


## Example 2: Adjusting Allele Frequencies

* Using estimated ancestry proportions $\hat{\pi}$ (potentially obtained using summix), we may update certain of the allele frequencies within a chosen observed ancestry, matching the continental ancestry proportions of a _target_ individual or sample.
    * For details and formulations, please refer to our paper.
* Here, we will use a $K=2$ ancestry example (1000 Genomes Indigenous Americans and 1000 Genomes African) observed within the 1000 Genomes African population.
    * The adjustment is performed on the 1000 Genomes African population.

* Below, we define $\pi_{\text{target}}$, the ancestry proportions from a chosen target sample
    * Here we assume our sample is 90% African and 10% Indigenous American
* We also define a new $\hat{\pi}$ (since we are modeling at 2 ancestry population this time), roughly based off of our solution to the first example in this notebook

In [7]:
pi_target = np.array((0.1,0.9))
new_pi_hat = np.array((0.006,0.994))

* In the cell below, we specify the required inputs and perform our example adjustment. In detail, we specify: 
    * the $K=2$ reference ancestries (their column names as strings)
    * the chosen observed population (its column name as a string)
    * $\pi_{\text{target}}$ (as a vector)
    * the file name of the data (as a string)
    * the file format (as a string -- options are 'tab' or 'csv')
    * $\hat{\pi}$ (as a vector)

In [8]:
my_adj_AF = adjAF(ref = ['ref_iam_1000G','ref_afr_1000G'],
                obs = "ref_afr_1000G",
                pi_target = pi_target,
                file_name = "example-data/example-data.txt",
                file_format = 'tab',
                pi_hat = new_pi_hat)

By default we print out the first 5 rows of the user-provided reference ancestries 
 
 that will be used in the calculation. Please check column names for correctness: 
 
 
    ref_iam_1000G
0         0.7093
1         0.0000
2         0.2442
3         0.5000
4         0.3372 
 
 
 We also print out the first 5 rows of the user-provided observed ancestry 
 
  to be used in the calculation. Please check column name for correctness: 
 
 
 0    0.541663
1    0.035714
2    0.120048
3    0.959325
4    0.801565
Name: ref_afr_1000G, dtype: float64 
 
 

Adjustment complete! 
 
 A data frame called new_DF has been created for the user. 
 
 new_DF contains the original, user-provided data frame, 
 
 concatenated with the adjusted allele frequencies (appended as the last column).


* In the cell above, we see the printout from running adjAF in this example problem. We print by default the used columns from the provided data and indicate that the adjustment is completed, telling the user how to access a new data frame containing the adjusted frequencies.

* In the cell below, we verify that a new column has been appended to our data frame (it will be the last column), containing the adjusted frequencies

In [9]:
my_adj_AF.head(10)

,CHR,RSID,BP,A1,A2,ref_eur_1000G,ref_afr_1000G,ref_sas_1000G,ref_eas_1000G,ref_iam_1000G,gnomAD_afr,gnomAD_amr,gnomAD_oth,adjusted_AF
0,1,rs2887286,1156131,C,T,0.173275,0.541663,0.531712,0.846223,0.7093,0.488610,0.525943,0.229705,0.5575164414486923
1,1,rs41477744,2329564,A,G,0.001238,0.035714,0.000000,0.000000,0.0000,0.045914,0.001179,0.008272,0.03233705794768612
2,1,rs9661525,2952840,G,T,0.168316,0.120048,0.099180,0.393853,0.2442,0.135977,0.286052,0.155617,0.13178892615694165
3,1,rs2817174,3044181,C,T,0.428213,0.959325,0.639072,0.570454,0.5000,0.854879,0.488180,0.470425,0.9158880605633803
4,1,rs12139206,3504073,T,C,0.204215,0.801565,0.393671,0.389881,0.3372,0.724178,0.295508,0.258748,0.7576516382293762
5,1,rs7514979,3654595,T,C,0.004951,0.418652,0.000000,0.000000,0.0000,0.336249,0.016509,0.024816,0.37906133269617703
6,1,rs9792946,4119662,G,A,0.100259,0.246031,0.170780,0.068450,0.0000,0.189055,0.058962,0.087477,0.22276462364185112
7,1,rs10753372,4273842,T,C,0.526005,0.821432,0.663622,0.699395,0.8605,0.777803,0.682033,0.641144,0.8251261501006038
8,1,rs678246,4561467,G,T,0.103961,0.141857,0.149279,0.001984,0.0000,0.148285,0.057783,0.106618,0.1284421527162978
9,1,rs12029491,5017652,T,C,0.040842,0.118063,0.073624,0.225200,0.0000,0.106782,0.024764,0.051471,0.10689763581488934
